## Problem Set 2: Merging and regular expressions

**Total points (without extra credit)**: 30 

**Background on the policy context**: here, we're going to use two datasets to practice reshaping, merging, and regular expression patterns. Both datasets relate to the broader issue of which employers might be violating the rights of temporary guestworkers granted visas under the H-2A program. Here are some articles about potential exploitation of guestworkers by firms and inequality caused by minimal oversight:

- News media coverage of labor abuses of temporary guestworkers: https://www.buzzfeednews.com/article/kenbensinger/the-pushovers 
- GAO report on labor abuses of temporary guestworkers: https://www.gao.gov/products/gao-15-154

The following datasets are located in `pset2_inputdata` (need to unzip): 

- `jobs_clean`: a dataset of guestworker jobs posted by many employers, some of whom have been debarred (banned) from the program for labor abuses; others not debarred
- `debar`: a dataset of employers who committed violations of labor regulations meant to protect temporary guestworkers 


You can view a codebook here: https://docs.google.com/spreadsheets/d/1rF9GJEC8pPKxipD0TsoG9DVdqz3EJ-b-BHEtyioAX7I/edit?usp=sharing


In [3]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [5]:
# Authors: Charlotte Sadelain, Hiba Ali, Farah Daredia
# QSS 20 PSET 2 
# Professo Chang
# 05/02/2025

# 1. Reshaping data (13 points total)

Load the following dataset stored in `pset2_inputdata`: `debar.csv`

This represents employers temporarily banned from hiring workers (debar.csv); call this `debar`


View the head()


In [9]:
import zipfile 
import csv

zip_path = "pset2_inputdata.zip"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall()

csv_name = "debar.csv"
df = pd.read_csv(csv_name) 

In [11]:
df.head()

,Name,"City, State",Violation,Duration,Start date,End date
0,J&J Harvesting,"Leads, ND",Failure to respond to audit (partial response),2 years,1/19/2014,1/18/2016
1,"Stahlman Apiaries, Inc","Selby, SD",Failure to respond to audit (partial response),1 year,2/19/2015,2/14/2016
2,Trust Nursery,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015
3,Anton Fertilizer Inc.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016
4,"Great Plains Fluid Service, Inc.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016


## 1.1 (1 point)

Print the number of rows in `debar` versus the number of unique employer names (`Name`). Is there one row per employer or multiple rows for some employers?

In [14]:
print( "Number of rows: ", df.shape[0])

print( "Number of unique employers:", df["Name"].nunique())

Number of rows:  114
Number of unique employers: 98


## 1.2 Investigating duplicated rows (2 points)

A. Create a new column in `debar`--`is_repeated`-- that tells us whether an employer (`Name`) is repeated > 1 times

*Hint*: there are multiple ways to solve this but some possibilities to get the list of names that are repeated are:
- Using value_counts() on the `Name` variable and extracting the index from that value counts 
- Using groupby to count the rows attached to one name

B. Print the rows where `is_repeated == True` and interpret

C. Subset to the rows where `is_repeated == True` and save that data as `mult_debar`. Print the head() and shape

In [17]:
counts = df.value_counts("Name")
repeated = counts[counts >1].index
df["is_repeated"] = (df["Name"].isin(repeated))

print(df[df["is_repeated"] == True])

mult_debar = (df[df["is_repeated"] == True])
mult_debar.head()
mult_debar.shape

                                        Name       City, State  \
6                    Annabella Land & Cattle     Annabella, UT   
7                        Autumn Hill Orchard        Groton, MA   
8      Caddo Creek Ranch, dba Paradise Ranch         Caddo, TX   
11                     Loewen Harvesting LLC   Brownsville, TX   
12               Rollo Farm Labor Contractor         Miami, FL   
14                             Sharon Mathis        Tifton, GA   
15                                 SRT Farms        Morton, TX   
16                               Mark Duncan     Roosevelt, UT   
17          Maple Ridge Custom Services, LLC     Altheimer, AK   
18                                 F&W Farms       Ingalls, KS   
19                        Cisco Produce Inc.         Cairo, GA   
21   Old Tree Farms/Verpaalen Custom Service         Volga, SD   
24               Rollo Farm Labor Contractor         Miami, FL   
25                     Loewen Harvesting LLC    Brownfield, TX   
28     Cad

,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,True
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,True
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,True
11,Loewen Harvesting LLC,"Brownsville, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,True
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,True


(32, 7)

## 1.3 Reshape mult_debar to wide to begin filtering out duplicates (4 points)

You want to separate out two cases:

- Cases where the repeat rows for one employer are due to duplicated data 
- Cases where the repeat rows for one employer represent repeated violations for different issues

There are various ways to check duplicates in this data (eg converting `Violation` to lowercase; replacing spelled-out states with two-dig state codes)

We're going to use the simple rule of:

- A row is a duplicate if, within an employer (defined by Name + City, State), the Start date for each row's violation is the same 

To begin to check this, reshape `mult_debar` to a wide dataframe (`mult_debar_wide`) with the following columns, treating the `Name` and `City, State` as the index for the pivot:

- Name
- City, State
- start_date_viol1
- start_date_viol2

Print the head and shape

In [20]:
mult_debar.loc[:,"counts"] = mult_debar.groupby(['Name', 'City, State']).cumcount() + 1

mult_debar_wide = mult_debar.pivot(index = ["Name", "City, State"], columns = "counts",values =  "Start date")
mult_debar_wide.columns = ['start_date_viol'+ str(i) for i in mult_debar_wide.columns]

mult_debar_wide.head()
mult_debar_wide.shape

/var/folders/8n/77bk6n_13svf1p2vh3dk7f8m0000gn/T/ipykernel_45452/563219045.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mult_debar.loc[:,"counts"] = mult_debar.groupby(['Name', 'City, State']).cumcount() + 1


start_date_viol1  \
Name                                  City, State                        
Annabella Land & Cattle               Annabella, UT           5/9/2014   
                                      Annabella, Utah         5/9/2014   
Autumn Hill Orchard                   Groton, MA              7/6/2014   
Caddo Creek Ranch, dba Paradise Ranch Caddo, TX              7/20/2014   
                                      Caddo, Texas           7/20/2014   

                                                      start_date_viol2  
Name                                  City, State                       
Annabella Land & Cattle               Annabella, UT                NaN  
                                      Annabella, Utah              NaN  
Autumn Hill Orchard                   Groton, MA              7/6/2014  
Caddo Creek Ranch, dba Paradise Ranch Caddo, TX                    NaN  
                                      Caddo, Texas                 NaN

(20, 2)

## 1.4 Filter out duplicates from original debar data (6 points)

A. Using `mult_debar_wide`, add a column `is_dup` that takes value of True for cases where start_date_viol1 == start_date_viol2 marking the row as a duplicate

B. Going back to the original long-format data you loaded at the beginning- `debar`
    - For employers where `is_dup == True` as indicated by your wide-format dataframe, only keep `violnum == viol1`
    - For all other employers (so is_dup == False and ones we didnt need to check duplicates for), keep all violnum
    - Remove the `is_repeated` column from the `debar` data

**Hint**: you can complete part B without a for loop; `pd.concat` with axis = 0 (row binding) is one way

Call the resulting dataframe `debar_clean` and print the shape and # of unique employer names

In [23]:
mult_debar_wide["is_dup"] = (mult_debar_wide["start_date_viol1"] == mult_debar_wide["start_date_viol2"])
merged = pd.merge(df, mult_debar_wide, on = ["Name", "City, State"], how = "left")


merged_sub1 = merged[merged['is_dup'] == True].drop_duplicates(subset=['Name', "City, State"], keep='first') 
merged_sub2 = merged[merged['is_dup'] != True]

debar_clean = pd.concat([merged_sub1, merged_sub2], axis=0)
debar_clean

print(debar_clean.shape)
print(debar_clean['Name'].nunique())

,Name,"City, State",Violation,Duration,Start date,End date,is_repeated,start_date_viol1,start_date_viol2,is_dup
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,True,7/6/2014,7/6/2014,True
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,True,8/23/2014,8/23/2014,True
14,Sharon Mathis,"Tifton, GA",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True,11/16/2014,11/16/2014,True
15,SRT Farms,"Morton, TX",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True,11/16/2014,11/16/2014,True
16,Mark Duncan,"Roosevelt, UT",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True,11/16/2014,11/16/2014,True
...,...,...,...,...,...,...,...,...,...,...
107,Walker Place,"Danville, IL",Failure to comply with the employer's obligati...,2 months,11/19/2019,1/26/2020,False,NaN,NaN,NaN
108,County Fair Farm (company) and Andrew Williams...,"Jefferson, ME",WHD Debarment,3 years,3/8/2017,3/8/2020,False,NaN,NaN,NaN
110,Jesus Ledesma,"Mulberry, FL",Failure to Respond to Audit Request,2 years,2/8/18,2/8/20,False,NaN,NaN,NaN
112,B & R Harvesting and Paul Cruz (individual),"Collins, Georgia",WHD Debarment,3 years,4/9/17,4/9/20,False,NaN,NaN,NaN


(105, 10)
98


In [45]:
mult_debar_wide["is_dup"] = np.where(mult_debar_wide.start_date_viol1 == mult_debar_wide.start_date_viol2, True, False)
mult_debar_wide

duplicates = mult_debar_wide[mult_debar_wide['is_dup'] == True]["Name"]

subset_dup = df[df.Name.isin(duplicates)]
subset_not_dup = df[~df.Name.isin(duplicates)]


subset_dup.shape
subset_dup.drop_duplicates("Name", inplace=True)
subset_dup.shape

debar_clean = pd.concat([subset_dup, subset_not_dup], axis=0)

debar_clean

,Name,"City, State",start_date_viol1,start_date_viol2,is_dup
0,69 Farms LLC,"Preston, Idaho",9/24/2016,9/24/2016,True
1,AB Ranch,"Stephenville, Texas",4/28/2017,4/28/2017,True
2,Abe- Nancy Froese,"Seminole, TX",11/23/2017,11/23/2017,True
3,Agecy I LLC,"Idabel, OK",8/27/2015,8/27/2015,True
4,"Agecy II, LLC","Ipswich, SD",3/25/2015,3/25/2015,True
...,...,...,...,...,...
96,Walker Place,"Danville, IL",11/19/2019,11/19/2019,True
97,Xavier Horne,"Lyons, Georgia",6/16/2016,9/27/2017,False
98,Yesenia Perez,"Axson, Georgia",9/27/2017,9/27/2017,True
99,Yolanda Chavez,"Santa Maria, CA",7/23/2014,7/23/2014,True


(107, 7)

/var/folders/8n/77bk6n_13svf1p2vh3dk7f8m0000gn/T/ipykernel_45452/436217641.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_dup.drop_duplicates("Name", inplace=True)


(94, 7)

,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
0,J&J Harvesting,"Leads, ND",Failure to respond to audit (partial response),2 years,1/19/2014,1/18/2016,False
1,"Stahlman Apiaries, Inc","Selby, SD",Failure to respond to audit (partial response),1 year,2/19/2015,2/14/2016,False
2,Trust Nursery,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015,False
3,Anton Fertilizer Inc.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016,False
4,"Great Plains Fluid Service, Inc.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016,False
...,...,...,...,...,...,...,...
39,Valley View Orchards,NaN,Impeding the Audit Process – Partial Response;,2 years,7/22/2015,7/21/2017,False
51,Old Tree Farms/Verpaalen Custom Service,"Volga, SD",Wage Hour Debarment,3 years,12/1/2014,12/1/2017,True
56,Cisco Produce Inc.,"Cairo, GA",Impeding the Audit Process – Non- Response,2 years,12/10/2015,12/9/2017,True
73,Xavier Horne,"Lyons, Georgia",Non-payment of certification fee,1 year,6/16/2016,6/15/2017,True


# 2. Merging and regex (17 points total)



## 2.1 Load data on job postings

The previous dataset contains a small subset of employers who faced temporary bans due to violations of H-2A program regulations

Since most of the bans have expired, we're going to see which of those employers posted new H-2A jobs in the first quarter of 2021 

Loading the `jobs_clean.csv` data stored in `pset2_inputdata`

In [8]:

csv_name2 = "jobs.csv"
df2 = pd.read_csv(csv_name2) 


##  2.2 Try inner join on employer name  (2 points)

- Use the `EMPLOYER_NAME` field of the `jobs` dataset
- Use the `Name` field of the `debar_clean` dataset 

A. Use pd.merge with an inner join on those fields to see whether there are any exact matches. 

B. If there are exact matches, print the row(s) with exact matches



In [9]:
names_joined = pd.merge(debar_clean, df2, left_on = "Name", right_on = "EMPLOYER_NAME", how = "inner")
names_joined

,Name,"City, State",Violation,Duration,Start date,End date,is_repeated,start_date_viol1,start_date_viol2,is_dup,...,ADDENDUM_B_HOUSING_ATTACHED,TOTAL_HOUSING_RECORDS,MEALS_PROVIDED,MEALS_CHARGED,MEAL_REIMBURSEMENT_MINIMUM,MEAL_REIMBURSEMENT_MAXIMUM,PHONE_TO_APPLY,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS
0,Rafael Barajas,"Sebring, Florida",Non-payment of certification fee,1 year,9/23/2016,9/22/2017,False,NaN,NaN,NaN,...,Y,3,Y,12.68,12.68,55.0,18632732686,NaN,https://seasonaljobs.dol.gov/,7


## 2.3 Targeted regex (10 points total)

You want to see if you can increase the exact match rate with some basic cleaning of each 
of the employer name fields in each dataset 

### 2.3.1 Converting to upper (2 points)

A. Convert the `EMPLOYER_NAME` and `Name` fields to uppercase using list comprehension rather than df.varname.str.upper() (it's fine to do a separate list comprehension line for each of the two columns)

B. Print a random sample of 15 values of each result

C. Assign the full vector of uppercase names back to the original data, writing over the original `EMPLOYER_NAME` and `Name` columns 


In [10]:
uppercase_debarclean = [name.upper() for name in debar_clean["Name"]]
uppercase_df2 = [name.upper() for name in df2["EMPLOYER_NAME"]]


In [11]:
random.sample(uppercase_debarclean, 15)
random.sample(uppercase_df2, 15)

['SRT FARMS',
 'MAPLE RIDGE CUSTOM SERVICES, LLC',
 'EVERGREEN PRODUCE',
 'XAVIER HORNE',
 'LESLIE COOK',
 'AGECY I LLC',
 'LOEWEN HARVESTING LLC',
 'PROMAX INC.',
 'YOLANDA CHAVEZ FARMING',
 'DANECO, LLC',
 'JIM AND ANN SHIPLEY WILLIAM SHIPLEY*',
 'NEMORIO RESENDIZ',
 'CHRIS ROBINSON',
 'MONICA SAAVEDRA (AGENT)',
 'MACKY AND BRAD FARMS']

['SESTER FARMS, INC',
 'FORKING PATHS VINEYARDS, INC.',
 'FORREST ARTHUR ',
 'SOUTHERN BELLE FARM, INC.',
 'WESTERN RANGE ASSOCIATION',
 'WIGGINS FARMS, LLC',
 'GILBERT ORCHARDS, INC.',
 'PRIME LINE INC.',
 'CHASEN GOAT GRAZING ',
 'OLSON CUSTOM HAYING LLC',
 'TEXAS BEST HONEY, INC.',
 'GLENN ECKELBERG',
 'MICHAEL N COMPTON',
 'INMAN HOLDING COMPANY ',
 'MONTY & CAROLENE FUNK']

In [12]:
debar_clean["Name"] = uppercase_debarclean
df2["EMPLOYER_NAME"] = uppercase_df2

### 2.3.2 Cleaning up punctuation (4 points)

You notice that INC, CO, and LLC are sometimes followed by a period (.) but sometimes not

A. For each dataset, write a regex pattern using `re.sub` to remove the . but only if it's preceded by INC, LLC, or CO 

Make sure LLC, INC, CO remain part of the string but just without the dot

B. Test the pattern on the positive and negative example we provide below and print the result. See the Github issue for examples of what to return


**Hint**: https://stackoverflow.com/questions/7191209/python-re-sub-replace-with-matched-content



In [13]:
pos_example_1 = "CISCO PRODUCE INC."
pos_example_2 = "AVOYELLES HONEY CO., LLC"
neg_example = "E.V. RANCH LLP"

In [14]:
pattern_match = r"\b(INC|CO|LLC)\."
replacement = r'\1'

clean_ex1 = re.sub(pattern_match, replacement, pos_example_1)
clean_ex2 = re.sub(pattern_match, replacement, pos_example_2)
clean_ex3 = re.sub(pattern_match, replacement, neg_example)

print(clean_ex1, clean_ex2, clean_ex3)


CISCO PRODUCE INC AVOYELLES HONEY CO, LLC E.V. RANCH LLP


### 2.3.3 (4 points)

Use that pattern in conjunction with `re.sub` and list comprehension to clean the employer name columns in each dataset. Save the new columns as `name_clean` in each. Then, use row subsetting to (1) subset to rows that changed names and (2) for:

- `debar_clean` print the `Name` and `name_clean` columns
- `jobs` print the `EMPLOYER_NAME` and `name_clean` columns

Make sure to use the uppercase versions of the variables


In [15]:
debar_clean["name_clean"] = [re.sub(pattern_match, replacement, name_str) for name_str in debar_clean.Name]
df2["name_clean"] = [re.sub(pattern_match, replacement, name_str) for name_str in df2.EMPLOYER_NAME]

debar_clean_sub = debar_clean[debar_clean["Name"] != debar_clean["name_clean"]]
df2_sub = df2[df2["EMPLOYER_NAME"] != df2["name_clean"]]

In [16]:
print(debar_clean_sub[['Name','name_clean']])
print(df2_sub[['EMPLOYER_NAME', 'name_clean']])

                                Name                       name_clean
3              ANTON FERTILIZER INC.             ANTON FERTILIZER INC
4   GREAT PLAINS FLUID SERVICE, INC.  GREAT PLAINS FLUID SERVICE, INC
5                        PROMAX INC.                       PROMAX INC
13                     REIMER'S INC.                     REIMER'S INC
19                CISCO PRODUCE INC.                CISCO PRODUCE INC
23                     REIMER’S INC.                     REIMER’S INC
33                GERONIMO SHEEP CO.                GERONIMO SHEEP CO
46          ALTENDORF TRANSPORT INC.          ALTENDORF TRANSPORT INC
56                CISCO PRODUCE INC.                CISCO PRODUCE INC
72      SAXTONS RIVER ORCHARDS, INC.      SAXTONS RIVER ORCHARDS, INC
91          AVOYELLES HONEY CO., LLC          AVOYELLES HONEY CO, LLC
                                 EMPLOYER_NAME  \
4                      DUNSON HARVESTING, INC.   
7     FARM LABOR ASSOCIATION FOR GROWERS, INC.   
14        

## 2.4 More joins and more cleaning (5 points)

A. Conduct another inner join between `jobs` and `debar_clean` now using the `name_clean` column; print the result. Did the cleaning result in any more employers matched between the two datasets?

B. Create a new column in `debar_clean` called `name_clean_2` that uses regex to take the following name in that dataset:

- `SLASH E.V. RANCH LLP` in the `debar_clean` dataset

And cleans it up so that it matches with this employer in `jobs`

- `SLASH EV RANCH` in the `jobs` dataset

Eg a pattern to remove the dots in the EV and the space+LLP-- you can apply the pattern to all employer names in debar_clean (so don't need to worry about only applying it to that one employer)


C. Conduct a left join using `name_clean_2` as the join column where the left hand dataframe is `jobs`; right hand dataframe is `debar_clean`, store the result as a dataframe, and print the rows where the merge indicator indicates the row was found in both dataframe

**Note**: this manual cleaning process is inefficient and helps motivate why talked about fuzzy matching. Fuzzy matching could recognize that Slash EV ranch is a highly similar string to slash ev ranch llp and match them without us needing to use regex to make the strings identical.

In [26]:
# Part A
names_joined2 = pd.merge(debar_clean, df2, left_on = "name_clean", right_on = "name_clean", how = "inner")
names_joined2

,Name,"City, State",Violation,Duration,Start date,End date,is_repeated,start_date_viol1,start_date_viol2,is_dup,...,ADDENDUM_B_HOUSING_ATTACHED,TOTAL_HOUSING_RECORDS,MEALS_PROVIDED,MEALS_CHARGED,MEAL_REIMBURSEMENT_MINIMUM,MEAL_REIMBURSEMENT_MAXIMUM,PHONE_TO_APPLY,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS
0,RAFAEL BARAJAS,"Sebring, Florida",Non-payment of certification fee,1 year,9/23/2016,9/22/2017,False,NaN,NaN,NaN,...,Y,3,Y,12.68,12.68,55.0,18632732686,NaN,https://seasonaljobs.dol.gov/,7


In [19]:
#Part B

def further_clean(name):
    name = re.sub(r'([A-Z])\.', r'\1', name)
    name = re.sub(r'( INC\.?| LLC\.?| LLP\.?| L\.L\.P\.?| CORP\.?| CORPORATION)', '', name)
    name = re.sub(r'\s+', ' ', name)
    return name.strip()

debar_clean["name_clean_2"] = [further_clean(name) for name in debar_clean['name_clean']]

In [24]:
# Part C
names_joined3 = pd.merge(df2, debar_clean, left_on = "name_clean", right_on = "name_clean_2", how = "left", indicator =True)

in_both = names_joined3[names_joined3['_merge'] == 'both']

print(in_both)

             CASE_NUMBER                           CASE_STATUS  \
791   H-300-20287-876656  Determination Issued - Certification   
1115  H-300-20306-894148  Determination Issued - Certification   

                RECEIVED_DATE            DECISION_DATE  \
791   2020-10-20 09:20:32.010  2020-11-09 00:00:00.000   
1115  2020-11-02 18:11:29.140  2020-11-24 00:00:00.000   

     TYPE_OF_EMPLOYER_APPLICATION H2A_LABOR_CONTRACTOR  \
791           Individual Employer                    Y   
1115          Individual Employer                    N   

     NATURE_OF_TEMPORARY_NEED EMERGENCY_FILING   EMPLOYER_NAME TRADE_NAME_DBA  \
791                  Seasonal                Y  RAFAEL BARAJAS            NaN   
1115                 Seasonal                N  SLASH EV RANCH            NaN   

      ... Duration  Start date    End date is_repeated start_date_viol1  \
791   ...   1 year   9/23/2016   9/22/2017       False              NaN   
1115  ...   1 year  11/15/2014  11/14/2015       False   

# 3. Optional extra credit 1: regex to separate companies from individuals (1 point)

You notice some employers in `debar_clean` have both the name of the company and the name of individual, e.g.:
    
COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*

Use the uppercase/cleaned `name_clean` in `debar_clean`

A. Write a regex pattern that does the following:
    - Captures the pattern that occurs before COMPANY if (COMPANY) is in string; so in example above, extracts COUNTY FAIR FARM 
    - Captures the pattern that occurs before INDIVIDUAL if (INDIVIDUAL) is also in string -- so in above, extracts ANDREW WILLIAMSON (so omit the "and")
    
B. Test the pattern on `pos_example` and `neg_example`-- make sure former returns a list (if using find.all) or match object (if using re.search) with the company name and individual name separated out; make sure latter returns empty
    
**Hints and resources**: for step A, you can either use re.search, re.match, or re.findall; don't worry about matching B&R Harvesting and Paul Cruz (Individual)

- Same regex resources as above
    

In [ ]:
pos_example = "COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*"
neg_example = "CISCO PRODUCE INC"

## your code here to define the pattern

## your code here to apply it to the pos_example

## your code here to apply it to the negative example

C. Iterate over the `name_clean` column in debar and use regex to create two new columns in `debar_clean`:
   - `co_name`: A column for company (full `name_clean` string if no match; pattern before COMPANY if one extracted)
   - `ind_name`: A column for individual (full `name_clean` string if no match; pattern before INDIVIDUAL if one extracted)
 


In [ ]:
# your code here

   
D. Print three columns for the rows in `debar_clean` containing the negative example and positive example described above (county fair farm and cisco produce):

- `name_clean`
- `co_name`
- `ind_name`
- `Violation`

**Note**: as shown in the outcome there may be duplicates of the same company reflecting different violations

In [ ]:
# your code here

# 4. Optional extra credit 2 (up to 3 points)

- For 1 point extra credit, create a visualization with 1+ of the existing fields in either the raw `jobs` or `debar` data. We'll be showing cool visualizations in class so use your imagination! Options could include visualizing between-state or over-time variation

- For 3 points extra credit instead, geocode the employer addresses in `jobs` and plot the addresses of jobs as points overlaid on top of a map of Georgia 
    - **Note**: this extra credit involves Googling since we have not yet covered spatial data. 
        - For discussion of how to geocode addresses -> lat/long, see: https://www.natasshaselvaraj.com/a-step-by-step-guide-on-geocoding-in-python/ 
        - For discussion of plotting lat/long dots against a map, see this discussion of geopandas: https://towardsdatascience.com/plotting-maps-with-geopandas-428c97295a73
    - Relevant columns include `EMPLOYER_ADDRESS_1` 
    - The geocoding might have a long runtime so feel free to implement it in a separate .py script that you submit alongside your notebook and to just read in the geocoded data

In [ ]:
## your code here